# Model I/O
> Source: [python.langchain.com/docs/modules/model_io](https://python.langchain.com/docs/modules/model_io/)

The core element of any language model application is...the model. LangChain gives you the building blocks to interface with any language model.

![Model I/O](https://python.langchain.com/assets/images/model_io-e6fc0045b7eae0377a4ddeb90dc8cdb8.jpg)

# Quickstart
The below quickstart will cover the basics of using LangChain's Model I/O components. It will introduce the two different types of models - LLMs and Chat Models. It will then cover how to use Prompt Templates to format the inputs to these models, and how to use Output Parsers to work with the outputs.

Language models in LangChain come in two flavors:

## ChatModels
[Chat models](https://python.langchain.com/docs/modules/model_io/chat/) are often backed by LLMs but tuned specifically for having conversations. Crucially, their provider APIs use a different interface than pure text completion models. Instead of a single string, they take a list of chat messages as input and they return an AI message as output. See the section below for more details on what exactly a message consists of. GPT-4 and Anthropic's Claude-2 are both implemented as chat models.

## LLMs
[LLMs](https://python.langchain.com/docs/modules/model_io/llms/) in LangChain refer to pure text completion models. The APIs they wrap take a string prompt as input and output a string completion. OpenAI's GPT-3 is implemented as an LLM.

These two API types have different input and output schemas.

Additionally, not all models are the same. Different models have different prompting strategies that work best for them. For example, Anthropic's models work best with XML while OpenAI's work best with JSON. You should keep this in mind when designing your apps.

In [1]:
from dotenv import load_dotenv

# make sure you have a .env file in the root directory
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI()
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

Both `llm` and `chat_model` are objects that represent configuration for a particular model. You can initialize them with parameters like `temperature` and others, and pass them around. The main difference between them is their input and output schemas. The LLM objects take string as input and output string. The ChatModel objects take a list of messages as input and output a message.

We can see the difference between an LLM and a ChatModel when we invoke it.

In [4]:
text = "What would be a good company name for a company that makes colorful socks?"

print(llm.invoke(text))



"Rainbow Threads Co."


In [5]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content=text)]
print(chat_model.invoke(messages))

content='Rainbow Footwear Co.' response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 22, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3aef089b-a0d4-4f45-8c71-42399a363345-0'


# Prompt Templates

Most LLM applications do not pass user input directly into an LLM. Usually they will add the user input to a larger piece of text, called a prompt template, that provides additional context on the specific task at hand.

For our application, it would be great if the user only had to provide the description of a company/product without worrying about giving the model instructions.

`PromptTemplates` help with exactly this! They bundle up all the logic for going from user input into a fully formatted prompt. This can start off very simple - for example, a prompt to produce the above string would just be:

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

There are several advantages of using these over raw string formatting. You can "partial" out variables - e.g. you can format only some of the variables at a time. You can compose them together, easily combining different templates into a single prompt. For explanations of these functionalities, see the [section on prompts](https://python.langchain.com/docs/modules/model_io/prompts/) for more detail.

`PromptTemplates` can also be used to produce a list of messages. In this case, the prompt not only contains information about the content, but also each message (its role, its position in the list, etc.). Here, what happens most often is a `ChatPromptTemplate` is a list of `ChatMessageTemplates`. Each `ChatMessageTemplate` contains instructions for how to format that `ChatMessage` - its role, and then also its content. Let's take a look at this below

In [11]:
from langchain_core.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

ChatPromptTemplates can also be constructed in other ways - see the [section on prompts](https://python.langchain.com/docs/modules/model_io/prompts/) for more detail.

# Output parsers
`OutputParsers` convert the raw output of a language model into a format that can be used downstream. There are a few main types of `OutputParsers`, including:
  - Convert text from `LLM` into structured information (e.g. JSON)
  - Convert a `ChatMessage` into just a string
  - Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

For full information on this, see the [section on output parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/).

In this getting started guide, we use a simple one that parses a list of comma separated values.

In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

# Composing with LCEL
We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to a language model, and then pass the output through an (optional) output parser. This is a convenient way to bundle up a modular piece of logic. Let's see it in action!

In [13]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
# use LCEL to create a chain
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "colors"})

['red', 'blue', 'green', 'yellow', 'purple']

Note that we are using the `|` syntax to join these components together. This `|` syntax is powered by the LangChain Expression Language (LCEL) and relies on the universal `Runnable` interface that all of these objects implement. To learn more about LCEL, read the documentation [here](https://python.langchain.com/docs/expression_language/).